In [1]:
import numpy as np
import tensorflow as tf
import h5py
import random

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 数据准备

In [2]:
#读取数据
f=h5py.File('D:/data/wind/tdata.h5','r')
data_x=f['x']
data_y=f['y']
#分配训练测试集
num=list(range(data_x.shape[0]))
num1=random.sample(num,200)
num2=set(num)-set(num1)
num2=list(num2)
num1.sort()


x_train=data_x[num2]
y_train=data_y[num2]

x_test=data_x[num1]
y_test=data_y[num1]
#生成批次
train_queue = tf.train.slice_input_producer([x_train,y_train],shuffle=None)
val_queue = tf.train.slice_input_producer([x_test,y_test],shuffle=None)
batch_xt,batch_yt=tf.train.shuffle_batch(train_queue,batch_size=16,capacity=500,min_after_dequeue=150)
batch_xv,batch_yv=tf.train.shuffle_batch(val_queue,batch_size=16,capacity=500,min_after_dequeue=150)

# 搭建模型

### 超参数设置

In [3]:
seq_length = batch_xt.shape[1]  # Time series will have the same past and future (to be predicted) lenght. 
batch_size = batch_xt.shape[0]  # Low value used for live demo purposes - 100 and 1000 would be possible too, crank that up!

output_dim = 1
input_dim = batch_xt.shape[-1]  # Output dimension (e.g.: multiple signals at once, tied in time)
hidden_dim = 10  # Count of hidden neurons in the recurrent units. 
layers_stacked_count = 3  # Number of stacked recurrent cells, on the neural depth axis. 

# Optmizer: 
learning_rate = 0.007  # Small lr helps not to diverge during training. 
nb_iters = 10000  # How many times we perform a training step (therefore how many times we show a batch). 
lr_decay = 0.92  # default: 0.9 . Simulated annealing.
momentum = 0.5  # default: 0.0 . Momentum technique in weights update
lambda_l2_reg = 0.003  # L2 regularization of weights - avoids overfitting

### 搭建s2s

In [4]:
with tf.variable_scope('Seq2seq'):

    # Encoder: inputs
    enc_inp = [
        tf.placeholder(tf.float32, shape=(None, input_dim), name="inp_{}".format(t))
           for t in range(seq_length)
    ]

    # Decoder: expected outputs
    expected_sparse_output = [
        tf.placeholder(tf.float32, shape=(None, output_dim), name="expected_sparse_output_".format(t))
          for t in range(seq_length)
    ]
    
    # Give a "GO" token to the decoder. 
    # Note: we might want to fill the encoder with zeros or its own feedback rather than with "+ enc_inp[:-1]"
    dec_inp = [ tf.zeros_like(enc_inp[0], dtype=np.float32, name="GO") ] + enc_inp[:-1]

    # Create a `layers_stacked_count` of stacked RNNs (GRU cells here). 
    cells = []
    for i in range(layers_stacked_count):
        with tf.variable_scope('RNN_{}'.format(i)):
            cells.append(tf.contrib.rnn.GRUCell(hidden_dim))
            # cells.append(tf.nn.rnn_cell.BasicLSTMCell(...))
    cell = tf.contrib.rnn.MultiRNNCell(cells)
    
    # Here, the encoder and the decoder uses the same cell, HOWEVER,
    # the weights aren't shared among the encoder and decoder, we have two
    # sets of weights created under the hood according to that function's def. 
    dec_outputs, dec_memory = tf.contrib.legacy_seq2seq.basic_rnn_seq2seq(
        enc_inp, 
        dec_inp, 
        cell
    )
    
    # For reshaping the output dimensions of the seq2seq RNN: 
    w_out = tf.Variable(tf.random_normal([hidden_dim, output_dim]))
    b_out = tf.Variable(tf.random_normal([output_dim]))
    
    # Final outputs: with linear rescaling for enabling possibly large and unrestricted output values.
    output_scale_factor = tf.Variable(1.0, name="Output_ScaleFactor")
    
    reshaped_outputs = [output_scale_factor*(tf.matmul(i, w_out) + b_out) for i in dec_outputs]

### 搭建损失函数 

In [5]:
# Training loss and optimizer

with tf.variable_scope('Loss'):
    # L2 loss
    output_loss = 0
    for _y, _Y in zip(reshaped_outputs, expected_sparse_output):
        output_loss += tf.reduce_mean(tf.nn.l2_loss(_y - _Y))
        
    # L2 regularization (to avoid overfitting and to have a  better generalization capacity)
    reg_loss = 0
    for tf_var in tf.trainable_variables():
        if not ("Bias" in tf_var.name or "Output_" in tf_var.name):
            reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))
            
    loss = output_loss + lambda_l2_reg * reg_loss
    tf.summary.scalar('loss',loss)

with tf.variable_scope('acc'):
    acc=0
    num=0
    for _y, _Y in zip(reshaped_outputs, expected_sparse_output):
        acc += tf.reduce_mean(tf.nn.l2_loss(_y - _Y))/tf.reduce_mean(tf.nn.l2_loss(_Y-0))
        num=num+1
    acc=acc/num
    acc=1-acc
    tf.summary.scalar('acc',acc)

with tf.variable_scope('Optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay=lr_decay, momentum=momentum)
    train_op = optimizer.minimize(loss)
    merged_summary = tf.summary.merge_all()

### 训练模型

In [6]:
min_loss=10000
sess=tf.Session()
train_writer=tf.summary.FileWriter('D:/graph/twomonthes/wind1/train/',sess.graph)
test_writer = tf.summary.FileWriter('D:/graph/twomonthes/wind1/test/', sess.graph)
saver=tf.train.Saver(max_to_keep=3)

init_op = tf.global_variables_initializer()
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)

for training_itr in range(10000):
    x1, y1 = sess.run([batch_xt,batch_yt])
    feed_dict1 = {enc_inp[t]: x1[:,t,:] for t in range(len(enc_inp))}
    feed_dict1.update({expected_sparse_output[k]: y1[:,k,:] for k in range(len(expected_sparse_output))})
    _, loss1,acc1,summaries1 = sess.run([train_op, loss,acc,merged_summary], feed_dict1)

    train_writer.add_summary(summaries1, training_itr)
    if training_itr %5==0:
#             saver.save(sess=sess, save_path='model/hand_landmark_v6.1_model/model.ckpt',global_step=(global_step + 1))
        mean_val_loss = 0

        x2,y2=sess.run([batch_xv,batch_yv])
        feed_dict2 = {enc_inp[t]: x2[:,t,:] for t in range(len(enc_inp))}
        feed_dict2.update({expected_sparse_output[t]: y2[:,t,:] for t in range(len(expected_sparse_output))})
        loss2,acc2,summaries2 = sess.run([loss,acc,merged_summary], feed_dict2)

        print('step: {}'.format(training_itr))
        print('train: loss: {} acc: {}  val: loss: {} acc: {}'.format(loss1,acc1,loss2,acc2))
        test_writer.add_summary(summaries2, training_itr)
        if loss1 < min_loss:
            min_loss=loss1
            saver.save(sess=sess, save_path='D:/model//twomonthes/wind1/model.ckpt',global_step=(training_itr + 1))
sess.close()
coord.request_stop()
    






step: 0
train: loss: 34476684.0 acc: -0.0028951168060302734  val: loss: 111415208.0 acc: 0.0031946301460266113
step: 5
train: loss: 21001980.0 acc: 0.03326618671417236  val: loss: 60552792.0 acc: 0.01745396852493286
step: 10
train: loss: 38419348.0 acc: 0.03302466869354248  val: loss: 25809712.0 acc: 0.04363095760345459
step: 15
train: loss: 48217588.0 acc: 0.03744661808013916  val: loss: 22910484.0 acc: 0.052390098571777344
step: 20
train: loss: 40999248.0 acc: 0.04620206356048584  val: loss: 77671336.0 acc: 0.030995547771453857
step: 25
train: loss: 37681016.0 acc: 0.057683587074279785  val: loss: 29605200.0 acc: 0.06563699245452881
step: 30
train: loss: 59751852.0 acc: 0.04483175277709961  val: loss: 36920576.0 acc: 0.06155937910079956
step: 35
train: loss: 45183564.0 acc: 0.06259810924530029  val: loss: 33717584.0 acc: 0.06251060962677002
step: 40
train: loss: 38245072.0 acc: 0.06138259172439575  val: loss: 16956422.0 acc: 0.10719603300094604
step: 45
train: loss: 43996880.0 acc: 0

step: 395
train: loss: 657596.875 acc: 0.8703495264053345  val: loss: 12843689.0 acc: 0.6107110977172852
step: 400
train: loss: 791167.9375 acc: 0.8167446851730347  val: loss: 9812804.0 acc: 0.6725301742553711
step: 405
train: loss: 12004883.0 acc: 0.5388555526733398  val: loss: 21943198.0 acc: 0.57313072681427
step: 410
train: loss: 457540.1875 acc: 0.8321945667266846  val: loss: 14565798.0 acc: 0.6498714089393616
step: 415
train: loss: 1697061.0 acc: 0.7756757736206055  val: loss: 26879370.0 acc: 0.574226975440979
step: 420
train: loss: 5320892.0 acc: 0.7487071752548218  val: loss: 21576758.0 acc: 0.5949835181236267
step: 425
train: loss: 2908818.25 acc: 0.8236672878265381  val: loss: 17367756.0 acc: 0.6008491516113281
step: 430
train: loss: 1752172.25 acc: 0.8509324193000793  val: loss: 39467256.0 acc: 0.4986134171485901
step: 435
train: loss: 1043892.875 acc: 0.8795480728149414  val: loss: 10743271.0 acc: 0.6727398633956909
step: 440
train: loss: 7152224.0 acc: 0.6864197850227356  

step: 795
train: loss: 13621606.0 acc: 0.8140578269958496  val: loss: 7752155.5 acc: 0.7840930819511414
step: 800
train: loss: 11016167.0 acc: 0.851718544960022  val: loss: 24159706.0 acc: 0.759017288684845
step: 805
train: loss: 10624907.0 acc: 0.810798168182373  val: loss: 14717858.0 acc: 0.7258673310279846
step: 810
train: loss: 6988831.5 acc: 0.8599867224693298  val: loss: 28224802.0 acc: 0.6758928298950195
step: 815
train: loss: 6207606.0 acc: 0.7778469920158386  val: loss: 31435044.0 acc: 0.6754424571990967
step: 820
train: loss: 6697622.0 acc: 0.6852985620498657  val: loss: 16261066.0 acc: 0.8241550922393799
step: 825
train: loss: 6733633.0 acc: 0.8863685131072998  val: loss: 30530534.0 acc: 0.6989288330078125
step: 830
train: loss: 8541499.0 acc: 0.8869803547859192  val: loss: 10085547.0 acc: 0.7516784071922302
step: 835
train: loss: 10998383.0 acc: 0.8809555768966675  val: loss: 3376403.5 acc: 0.9150993227958679
step: 840
train: loss: 7327928.5 acc: 0.927736759185791  val: los

step: 1195
train: loss: 13263848.0 acc: 0.6836504936218262  val: loss: 11498497.0 acc: 0.7481537461280823
step: 1200
train: loss: 1596824.375 acc: 0.8771691918373108  val: loss: 11988598.0 acc: 0.7967002987861633
step: 1205
train: loss: 8044863.5 acc: 0.8425007462501526  val: loss: 30371768.0 acc: 0.6689443588256836
step: 1210
train: loss: 3015543.25 acc: 0.8434683084487915  val: loss: 8056881.0 acc: 0.8442305326461792
step: 1215
train: loss: 6731596.5 acc: 0.85820472240448  val: loss: 6732558.0 acc: 0.7318225502967834
step: 1220
train: loss: 8916610.0 acc: 0.7350637912750244  val: loss: 12781706.0 acc: 0.7900791764259338
step: 1225
train: loss: 5195051.0 acc: 0.8322274684906006  val: loss: 12170383.0 acc: 0.8001750111579895
step: 1230
train: loss: 6460771.0 acc: 0.810108482837677  val: loss: 11669407.0 acc: 0.8059514760971069
step: 1235
train: loss: 5614684.0 acc: 0.7771059274673462  val: loss: 18660188.0 acc: 0.7598909139633179
step: 1240
train: loss: 11514824.0 acc: 0.81438255310058

step: 1610
train: loss: 11514569.0 acc: 0.8695435523986816  val: loss: 2095472.25 acc: 0.9496012330055237
step: 1615
train: loss: 6511639.5 acc: 0.8888038396835327  val: loss: 26234098.0 acc: 0.7486158609390259
step: 1620
train: loss: 6459760.5 acc: 0.8782281875610352  val: loss: 13265274.0 acc: 0.7496618628501892
step: 1625
train: loss: 16337252.0 acc: 0.840154230594635  val: loss: 6569499.0 acc: 0.901332437992096
step: 1630
train: loss: 23932536.0 acc: 0.8677400350570679  val: loss: 20631230.0 acc: 0.6378713846206665
step: 1635
train: loss: 35218724.0 acc: 0.8470367193222046  val: loss: 3970798.5 acc: 0.8660362362861633
step: 1640
train: loss: 14267904.0 acc: 0.8738330602645874  val: loss: 8903256.0 acc: 0.8505712747573853
step: 1645
train: loss: 15855118.0 acc: 0.904881477355957  val: loss: 23428814.0 acc: -0.31775856018066406
step: 1650
train: loss: 15681212.0 acc: 0.9094858169555664  val: loss: 11743638.0 acc: 0.8699479699134827
step: 1655
train: loss: 5444466.0 acc: 0.85849022865

...

step: 8020
train: loss: 11028339.0 acc: 0.6933443546295166  val: loss: 2881141.0 acc: 0.965437650680542
step: 8025
train: loss: 9790639.0 acc: 0.704851508140564  val: loss: 12991031.0 acc: 0.6582731008529663
step: 8030
train: loss: 3175706.5 acc: 0.817805826663971  val: loss: 24113550.0 acc: 0.669499933719635
step: 8035
train: loss: 3532993.75 acc: 0.809001088142395  val: loss: 1537754.875 acc: 0.9022629261016846
step: 8040
train: loss: 3771790.75 acc: 0.7639550566673279  val: loss: 7543195.5 acc: 0.7125356197357178
step: 8045
train: loss: 8525349.0 acc: 0.7534374594688416  val: loss: 6956544.0 acc: 0.7961491346359253
step: 8050
train: loss: 8252529.5 acc: 0.8147412538528442  val: loss: 4536484.0 acc: 0.5598137378692627
step: 8055
train: loss: 8957078.0 acc: 0.852240800857544  val: loss: 24533224.0 acc: 0.65425705909729
step: 8060
train: loss: 4210082.0 acc: 0.9293858408927917  val: loss: 19944304.0 acc: 0.431765079498291
step: 8065
train: loss: 3383224.5 acc: 0.9414645433425903  val: 

step: 8425
train: loss: 6018990.5 acc: 0.9438116550445557  val: loss: 7103960.0 acc: 0.6208750009536743
step: 8430
train: loss: 7229030.5 acc: 0.9138950705528259  val: loss: 8807675.0 acc: 0.6995559930801392
step: 8435
train: loss: 3008793.0 acc: 0.893964946269989  val: loss: 3665100.0 acc: 0.9218642115592957
step: 8440
train: loss: 2651222.25 acc: 0.8542944192886353  val: loss: 1813748.375 acc: 0.8493852615356445
step: 8445
train: loss: 6488796.5 acc: 0.8313081860542297  val: loss: 7653338.5 acc: 0.8879546523094177
step: 8450
train: loss: 6822611.0 acc: 0.8225452899932861  val: loss: 4286071.5 acc: 0.8270743489265442
step: 8455
train: loss: 2836849.0 acc: 0.8863772749900818  val: loss: 2822071.75 acc: 0.8217534422874451
step: 8460
train: loss: 4182358.75 acc: 0.8597816228866577  val: loss: 14140464.0 acc: 0.6468740105628967
step: 8465
train: loss: 21530066.0 acc: -0.7115612030029297  val: loss: 6038980.0 acc: 0.6937140226364136
step: 8470
train: loss: 4632042.0 acc: 0.792204737663269 

step: 8830
train: loss: 3116348.0 acc: 0.9356578588485718  val: loss: 10955187.0 acc: 0.8614906072616577
step: 8835
train: loss: 3310558.0 acc: 0.9262933731079102  val: loss: 3985973.5 acc: 0.9344712495803833
step: 8840
train: loss: 3595083.5 acc: 0.9236022233963013  val: loss: 10438408.0 acc: 0.874589741230011
step: 8845
train: loss: 3059674.5 acc: 0.9525442719459534  val: loss: 20920036.0 acc: 0.244958758354187
step: 8850
train: loss: 1453890.875 acc: 0.97849440574646  val: loss: 841599.5625 acc: 0.9809091687202454
step: 8855
train: loss: 1857622.5 acc: 0.9628384709358215  val: loss: 8603032.0 acc: 0.5720899105072021
step: 8860
train: loss: 7242504.5 acc: 0.8544045090675354  val: loss: 11757528.0 acc: 0.85066819190979
step: 8865
train: loss: 5829299.0 acc: 0.90827476978302  val: loss: 8369265.0 acc: 0.8014752864837646
step: 8870
train: loss: 6417270.0 acc: 0.8343323469161987  val: loss: 17002362.0 acc: 0.7256829738616943
step: 8875
train: loss: 5986767.0 acc: 0.9136295318603516  val:

step: 9230
train: loss: 422032.90625 acc: 0.9316386580467224  val: loss: 18543618.0 acc: 0.5148207545280457
step: 9235
train: loss: 500749.6875 acc: 0.9837260246276855  val: loss: 17966130.0 acc: 0.717068076133728
step: 9240
train: loss: 1872165.5 acc: 0.9083181619644165  val: loss: 20584876.0 acc: 0.7317209839820862
step: 9245
train: loss: 970947.5 acc: 0.919842004776001  val: loss: 11348968.0 acc: 0.7752582430839539
step: 9250
train: loss: 1226921.625 acc: 0.9433889389038086  val: loss: 11158287.0 acc: 0.7354850769042969
step: 9255
train: loss: 673363.625 acc: 0.9703154563903809  val: loss: 14032943.0 acc: 0.3993566632270813
step: 9260
train: loss: 667385.5625 acc: 0.9451054930686951  val: loss: 14551610.0 acc: 0.7145801782608032
step: 9265
train: loss: 838833.3125 acc: 0.9080603718757629  val: loss: 30941542.0 acc: 0.7121970653533936
step: 9270
train: loss: 834694.25 acc: 0.9383665919303894  val: loss: 24486620.0 acc: 0.696147084236145
step: 9275
train: loss: 662819.625 acc: 0.94270

step: 9645
train: loss: 4817126.5 acc: 0.9357168674468994  val: loss: 7352888.5 acc: 0.761263906955719
step: 9650
train: loss: 2418766.5 acc: 0.9652509689331055  val: loss: 25263948.0 acc: 0.6991649866104126
step: 9655
train: loss: 1619022.25 acc: 0.9846481084823608  val: loss: 5910029.5 acc: 0.9224178194999695
step: 9660
train: loss: 6856208.0 acc: 0.9361594319343567  val: loss: 7272671.0 acc: 0.7815593481063843
step: 9665
train: loss: 2311577.0 acc: 0.9608935713768005  val: loss: 8638782.0 acc: 0.8631575703620911
step: 9670
train: loss: 3694923.0 acc: 0.9445474743843079  val: loss: 8728517.0 acc: 0.8404074907302856
step: 9675
train: loss: 465078.875 acc: 0.9941495060920715  val: loss: 10386895.0 acc: 0.8094267249107361
step: 9680
train: loss: 1505646.125 acc: 0.970809817314148  val: loss: 11564161.0 acc: 0.8161998391151428
step: 9685
train: loss: 5604896.0 acc: 0.7877157926559448  val: loss: 8990937.0 acc: 0.8577907085418701
step: 9690
train: loss: 386757.375 acc: 0.9819884300231934 